In [1]:
from MS2LDA.motif_parser import load_m2m_folder
from MS2LDA.Add_On.MassQL.MassQL4MotifDB import motifs2motifDB
from MS2LDA.Add_On.MassQL.MassQL4MotifDB import store_motifDB_excel

In [2]:
import os

In [3]:
old_motifDB_path = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\pySubstructures\pySubstructures\resources\MOTIFDB"

In [4]:
old_motifDB_librarynames = os.listdir(old_motifDB_path)
old_motifDB_librarynames

['Euphorbia Plant Mass2Motifs',
 'GNPS library derived Mass2Motifs',
 'LDB MotifDB POS',
 'LDB_NEG_MotifDB_01',
 'LDB_NEG_MotifDB_02',
 'Massbank library derived Mass2Motifs',
 'MIADB_pos_100',
 'MIADB_pos_60',
 'MIADB_pos_indole',
 'Photorhabdus and Xenorhabdus Mass2Motifs',
 'Planomonospora-associated Mass2Motifs',
 'Rhamnaceae Plant Mass2Motifs',
 'Streptomyces and Salinispora Mass2Motifs',
 'Streptomyces S29',
 'Urine derived Mass2Motifs']

In [18]:
motifDB_motifs = []
negative_mode_libraries = ['LDB_NEG_MotifDB_01', 'LDB_NEG_MotifDB_02', 'Euphorbia Plant Mass2Motifs']
for libraryname in old_motifDB_librarynames:
    motifs = load_m2m_folder(old_motifDB_path + "/" + libraryname)
    for motif in motifs:
        motif.set("motifset", libraryname)
        if libraryname in negative_mode_libraries:
            motif.set("charge", -1)
        else:
            motif.set("charge", 1)
    motifDB_motifs += motifs

In [19]:
motifDB_new_format_ms1, motifDB_new_format_ms2 = motifs2motifDB(motifDB_motifs)

In [20]:
motifDB_new_format_ms1

,frag_mz,frag_intens,loss_mz,loss_intens,charge,ms2accuracy,short_annotation,annotation,motif_id,motifset,scan,ms1scan
0,NaN,NaN,168.08,0.003464,1,None,Paracetamol related,Paracetamol related,motif_97,Urine derived Mass2Motifs,149dc6a2a0bc6dc97cbb4da0694af393,0


In [21]:
motifDB_new_format_ms2

,frag_mz,frag_intens,loss_mz,loss_intens,charge,ms2accuracy,short_annotation,annotation,motif_id,motifset,scan,ms1scan
0,71.05,0.076913,NaN,NaN,-1,None,173 415 216 185 351 171 593 387,Unknown,motif_0,Euphorbia Plant Mass2Motifs,9e44605547a2c3c0a91d9a0fd1bbf816,0
1,97.03,0.130360,NaN,NaN,-1,None,173 415 216 185 351 171 593 387,Unknown,motif_0,Euphorbia Plant Mass2Motifs,9e44605547a2c3c0a91d9a0fd1bbf816,0
2,105.07,0.039002,NaN,NaN,-1,None,173 415 216 185 351 171 593 387,Unknown,motif_0,Euphorbia Plant Mass2Motifs,9e44605547a2c3c0a91d9a0fd1bbf816,0
3,109.06,0.096132,NaN,NaN,-1,None,173 415 216 185 351 171 593 387,Unknown,motif_0,Euphorbia Plant Mass2Motifs,9e44605547a2c3c0a91d9a0fd1bbf816,0
4,113.06,0.051623,NaN,NaN,-1,None,173 415 216 185 351 171 593 387,Unknown,motif_0,Euphorbia Plant Mass2Motifs,9e44605547a2c3c0a91d9a0fd1bbf816,0
...,...,...,...,...,...,...,...,...,...,...,...,...
56887,NaN,NaN,97.03,0.007884,1,None,Paracetamol related,Paracetamol related,motif_97,Urine derived Mass2Motifs,149dc6a2a0bc6dc97cbb4da0694af393,0
56888,NaN,NaN,121.12,0.003699,1,None,Paracetamol related,Paracetamol related,motif_97,Urine derived Mass2Motifs,149dc6a2a0bc6dc97cbb4da0694af393,0
56889,NaN,NaN,139.03,0.003818,1,None,Paracetamol related,Paracetamol related,motif_97,Urine derived Mass2Motifs,149dc6a2a0bc6dc97cbb4da0694af393,0
56890,NaN,NaN,139.99,0.002651,1,None,Paracetamol related,Paracetamol related,motif_97,Urine derived Mass2Motifs,149dc6a2a0bc6dc97cbb4da0694af393,0
